# PRACA DOMOWA 2

Plik *data.csv* zawiera dane medyczne Indian z plemienia Pima. Ostatnia kolumna (*Outcome*) zawiera binarną zmienną odpowiedzi, gdzie 1 oznacza, że dana osoba ma cukrzyce.

In [4]:
import pandas as pd
import numpy as np
import sklearn

data = pd.read_csv("data.csv")
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72.0,35,0,33.6,0.627,50.0,1
1,1,85,NaN,29,0,NaN,0.351,31.0,0
2,8,183,64.0,0,0,NaN,0.672,NaN,1
3,1,89,66.0,23,94,NaN,0.167,NaN,0
4,0,137,NaN,35,168,43.1,2.288,33.0,1


### Zadanie 0 Podziel zbiór na testowy i treningowy w proporcjach 0.3 / 0.7

In [5]:
train_mask = np.random.rand(len(data)) < 0.7
df_train = data[train_mask]
df_test = data[~train_mask]

## 1. Preprocessing

Wczytany zbiór danych posiada braki danych. Większość klasyfikatorów nie jest w stanie operować na tak wybrakowanych danych.


*uwaga*: drzewa decyzyjne (używane później) potrafią obsługiwać braki danych za pomocą *podziałów zastępczych*

### Zadanie 1.1


Wypełnij brakujące dane  
a) średnią  
b) medianą  
c) modą   
+d) przy pomocy metody Multivariate feature imputation z biblioteki *sklearn*  

Wyświetl 5 pierwszych wierszy przy pomocy metody *.head()*

In [6]:
#a)

df_train_mean = df_train.fillna(df_train.mean())
df_test_mean = df_test.fillna(df_train.mean())
df_train_mean.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
1,1,85,68.836502,29,0,31.750179,0.351,31.000000,0
2,8,183,64.000000,0,0,31.750179,0.672,33.962838,1
7,10,115,0.000000,0,0,35.300000,0.134,33.962838,0
8,2,197,70.000000,45,543,30.500000,0.158,53.000000,1
9,8,125,68.836502,0,0,0.000000,0.232,33.962838,1


In [7]:
#b)

df_train_med = df_train.fillna(df_train.median())
df_test_med = df_test.fillna(df_train.median())
df_train_med.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
1,1,85,72.0,29,0,32.0,0.351,31.0,0
2,8,183,64.0,0,0,32.0,0.672,30.0,1
7,10,115,0.0,0,0,35.3,0.134,30.0,0
8,2,197,70.0,45,543,30.5,0.158,53.0,1
9,8,125,72.0,0,0,0.0,0.232,30.0,1


In [8]:
#c)

df_train_mod = df_train.fillna(df_train.mode().iloc[0])
df_test_mod = df_test.fillna(df_train.mode().iloc[0])
df_train_mod.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
1,1,85,78.0,29,0,30.1,0.351,31.0,0
2,8,183,64.0,0,0,30.1,0.672,22.0,1
7,10,115,0.0,0,0,35.3,0.134,22.0,0
8,2,197,70.0,45,543,30.5,0.158,53.0,1
9,8,125,78.0,0,0,0.0,0.232,22.0,1


In [9]:
#d)
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer


imp = IterativeImputer(max_iter=10, random_state=0)
imp.fit(df_train)

df_train_imp = pd.DataFrame(imp.transform(df_train))
df_train_imp.columns = data.columns
df_test_imp = pd.DataFrame(imp.transform(df_test))
df_test_imp.columns = data.columns
df_test_imp.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6.0,148.0,72.000000,35.0,0.0,33.600000,0.627,50.000000,1.0
1,1.0,89.0,66.000000,23.0,94.0,29.152453,0.167,24.330455,0.0
2,0.0,137.0,97.134910,35.0,168.0,43.100000,2.288,33.000000,1.0
3,5.0,116.0,64.969083,0.0,0.0,27.239637,0.201,36.813351,0.0
4,3.0,78.0,50.000000,32.0,88.0,22.833798,0.248,26.000000,1.0


## Ocena jakości modelu

### Zadanie 2.1

Podziel zbior na testowy i treningowy. Wytrenuj drzewo decyzyjne na zbiorze z punktu a) zadania 1.1 (ustaw wartość 'max_depth' równą 5). Wyrysuj otrzymane drzewo korzystając z kodu w następnej komórce podstawiając swoje zmienne. 

In [10]:
from sklearn import tree

X_train = df_train_mean[data.columns[:-1]]
y_train = df_train_mean[data.columns[-1]]
X_test = df_test_mean[data.columns[:-1]]
y_test = df_test_mean[data.columns[-1]]

clf = tree.DecisionTreeClassifier(max_depth=5)
clf = clf.fit(X_train, y_train)


In [11]:
import matplotlib.pyplot as plt
plt.figure(figsize=(32, 18))
tree.plot_tree(clf, filled=True, feature_names=df_train_med.columns, class_names=("0", "1"))
plt.show()

<Figure size 3200x1800 with 1 Axes>

### Zadanie 2.2

Dla stworzonego drzewa oblicz  
a) Accuracy  
b) Error Rate  
c) Precision, Recall (True Positive Rate)
d) False Positive Rate


Wyrzuc ze zbioru testowego 90% wierszy z klasą 1. Porównaj Recall z Accuracy.  

Czy Accuracy jest dobrą metryką dla niezrównoważonych danych?  
Która z tych metryk jest najważniejsza w przypadku przewidywania czy dana osoba jest chora?

In [12]:
y_pred = clf.predict(X_test)

accuracy = sum(y_pred == y_test) / len(y_test)
print(f"Accuracy: {accuracy:.3f}")

error_rate = 1 - accuracy
print(f"Error Rate: {error_rate:.3f}")

precision = sum((y_pred == 1) & (y_pred == y_test)) / sum(y_pred)
recall = sum((y_pred == 1) & (y_pred == y_test)) / sum(y_test)
print(f"Precision: {precision:.3f}, Recall: {recall:.3f}")

fpr = 1 - recall
print(f"False Positive Rate: {fpr:.3f}")


#dla obciętego zbioru
test_0_idx = y_test == 0
test_1_idx = y_test == 1
trim_mask = np.random.rand(sum(test_1_idx)) < 0.3

X_test_trimmed = pd.concat([X_test[test_0_idx], (X_test[test_1_idx][trim_mask])])
y_test_trimmed = pd.concat([y_test[test_0_idx], (y_test[test_1_idx][trim_mask])])

y_pred_trimmed = clf.predict(X_test_trimmed)

accuracy_trimmed = sum(y_pred_trimmed == y_test_trimmed) / len(y_test_trimmed)
recall_trimmed = sum((y_pred_trimmed == 1) & (y_pred_trimmed == y_test_trimmed)) / sum(y_test_trimmed)
precision_trimmed = sum((y_pred_trimmed == 1) & (y_pred_trimmed == y_test_trimmed)) / sum(y_pred_trimmed)
print()
print(f"Accuracy on trimmed dataset: {accuracy_trimmed:.3f}")
print(f"Recall on trimmed dataset: {recall_trimmed:.3f}")
print(f"Precision on trimmed dataset: {precision_trimmed:.3f}")

Accuracy: 0.679
Error Rate: 0.321
Precision: 0.486, Recall: 0.538
False Positive Rate: 0.462

Accuracy on trimmed dataset: 0.719
Recall on trimmed dataset: 0.500
Precision on trimmed dataset: 0.178


### Zadanie 2.3

Na zbiorze z Zad 1.1a Zmierz Accuracy dla drzew wytrenowanych z wartościami parametru 'max_depth' 3, 5, 10 oraz 15.

Zmierz Accuracy metodą K-Fold Cross Validation (przy K = 5), w każdym 'obrocie pętli' imputując brakujące wartości średnią z aktualnego zbioru treningowego dla wartości 'max_depth' 3, 5, 10 oraz 15.

Porównaj wyniki obu metod, przy metodzie Cross Validation zwróć uwagę na odchylenia standardowa (metoda 'wyprodukuje' 5 wartości Accuracy dla każdego 'max_depth').

In [14]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score # nie chce nam sie liczyć tego ręcznie drugi raz :)

max_depths = [3, 5, 10, 15, 20]

accuracies_vanilla = []
for depth in max_depths:
    clf = tree.DecisionTreeClassifier(max_depth=depth)
    clf = clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_pred, y_test)
    accuracies_vanilla.append(accuracy)

accuracies_cv = []
for depth in max_depths:
    accuracies_depth = []
    folds = KFold(n_splits=5).split(df_train)
    for train_idx, test_idx in folds: 
        df_train_fold = data.iloc[train_idx]
        df_test_fold = data.iloc[test_idx]
        df_train_fold = df_train_fold.fillna(df_train_fold.mean())
        df_test_fold = df_test_fold.fillna(df_train_fold.mean())
        
        X_train_fold = df_train_fold[data.columns[:-1]]
        y_train_fold = df_train_fold[data.columns[-1]]
        X_test_fold = df_test_fold[data.columns[:-1]]
        y_test_fold = df_test_fold[data.columns[-1]]
        
        clf = tree.DecisionTreeClassifier(max_depth=depth)
        clf = clf.fit(X_train_fold, y_train_fold)
        y_pred_fold = clf.predict(X_test_fold)
        accuracy = accuracy_score(y_pred_fold, y_test_fold)
        accuracies_depth.append(accuracy)
    accuracies_cv.append(accuracies_depth)

cv_means_stds = [(np.mean(accuracies), np.std(accuracies)) for accuracies in accuracies_cv]
    
for depth, accuracy in zip(max_depths, accuracies_vanilla):
    print(f"max_depth: {depth}, accuracy: {accuracy:.3f}")
    
print()

for depth, accuracies in zip(max_depths, accuracies_cv):
    print(f"max_depth: {depth}, accuracy-cv-mean: {np.mean(accuracies):.3f}, accuracy-cv-std: {np.std(accuracies):.3f}")

max_depth: 3, accuracy: 0.684
max_depth: 5, accuracy: 0.679
max_depth: 10, accuracy: 0.670
max_depth: 15, accuracy: 0.651
max_depth: 20, accuracy: 0.684

max_depth: 3, accuracy-cv-mean: 0.723, accuracy-cv-std: 0.062
max_depth: 5, accuracy-cv-mean: 0.709, accuracy-cv-std: 0.052
max_depth: 10, accuracy-cv-mean: 0.667, accuracy-cv-std: 0.022
max_depth: 15, accuracy-cv-mean: 0.642, accuracy-cv-std: 0.037
max_depth: 20, accuracy-cv-mean: 0.637, accuracy-cv-std: 0.017


### Zadanie 2.4 (dodatkowe)
Powtórz kroki z zadania 2.3 dla pozostałych metod imputacji danych z zadania 1.1